# Assignment: LoRA-based Adapter Training for a Small GPT Model

## Objective
This assignment focuses on implementing and running Low-Rank Adaptation (LoRA) for fine-tuning a small Generative Pre-trained Transformer (GPT) model on a custom dataset. You will learn how LoRA enables efficient adaptation of large language models by training only a small number of new parameters, significantly reducing computational costs and memory requirements compared to full fine-tuning.

## Part 1: Environment Setup and Dataset Preparation (30 Marks)

1.  **Environment Setup:**
    * Create a new Python virtual environment.
    * Install the necessary libraries: `transformers`, `torch`, `peft` (Parameter-Efficient Fine-Tuning library), `datasets`, `accelerate` (for distributed training setup, even if running on a single GPU), `numpy`, `scikit-learn`.
        * Provide a `requirements.txt` file listing all dependencies.

2.  **Dataset Acquisition and Preprocessing:**
    * Select a text dataset suitable for a generative task (e.g., text summarization, dialogue generation, question answering, creative writing).
        * **Recommended:** A small, focused dataset to see the effects of LoRA quickly. Examples:
            * A subset of a dialogue dataset (e.g., DailyDialog).
            * A small collection of simple story prompts and completions.
            * Custom QA pairs where the answer is short and direct.
    * **Minimum Requirement:** The dataset should contain at least **500 pairs/samples** for training, where each sample is a text input that you want the model to generate a continuation for. Ensure your data is in a format like `{'text': 'input text for generation'}`.
    * Load your chosen dataset using the `datasets` library (e.g., `load_dataset` if public, or `load_dataset('text', data_files={'train': 'train.txt', 'test': 'test.txt'})`).
    * Describe your chosen dataset, its source, and its characteristics (e.g., average length of input/output, number of samples).

3.  **Tokenization:**
    * Load a tokenizer for a small GPT-like model (e.g., `gpt2`, `distilgpt2`, `openai-community/gpt2-small`).
    * Tokenize your dataset. For generative tasks, you typically concatenate input and target for causal language modeling.
        * Handle padding and truncation appropriately (`max_length` should be reasonable, e.g., 128 or 256).
        * Set `return_tensors="pt"`.
        * Ensure `labels` are set to `input_ids` for causal language modeling, masking special tokens if necessary (though `DataCollatorForLanguageModeling` usually handles this).
    * Print a sample of the tokenized input IDs and their corresponding decoded text to verify correctness.

In [ ]:
# Your code for environment setup, dataset loading, preprocessing, and tokenization here.
# Include dataset description and verification of tokenized data.

## Part 2: LoRA Configuration and Model Preparation (30 Marks)

1.  **Load Base GPT Model:**
    * Load a small pre-trained GPT model for causal language modeling (e.g., `AutoModelForCausalLM.from_pretrained("gpt2")` or `"distilgpt2"`).
    * Ensure the model is moved to your available device (CPU/GPU).
    * Print the model's structure to understand its layers.

2.  **LoRA Configuration with PEFT:**
    * Define a `LoraConfig` object from the `peft` library.
    * Key parameters to set and justify:
        * `r` (LoRA rank, e.g., 8, 16, 32): The rank of the update matrices. Explain its impact.
        * `lora_alpha` (LoRA scaling factor, e.g., 16, 32): Scales the LoRA weights. Explain its purpose.
        * `target_modules` (e.g., `["q_proj", "v_proj"]`): Which layers in the GPT model to apply LoRA to. Justify your choice based on common practices.
        * `lora_dropout` (e.g., 0.1): Dropout probability for LoRA layers.
        * `bias` (e.g., "none"): Whether to train bias parameters.
    * Explain what each chosen parameter signifies in the context of LoRA.

3.  **Prepare Model for PEFT Training:**
    * Use `peft_model = get_peft_model(model, peft_config)` to wrap your base GPT model with the LoRA adapters.
    * Print the trainable parameters of the `peft_model` using `peft_model.print_trainable_parameters()`.
    * Compare the number of trainable parameters in the LoRA-wrapped model to the total parameters of the base model. Discuss the significant reduction in trainable parameters and its implications.

In [ ]:
# Your code for loading the base GPT model, defining LoraConfig, and preparing the model with PEFT.
# Include justifications for LoRA parameters and a comparison of trainable parameters.

## Part 3: Training and Evaluation (40 Marks)

1.  **Define Training Arguments:**
    * Use `TrainingArguments` from `transformers`.
    * Set parameters such as:
        * `output_dir`
        * `num_train_epochs` (e.g., 3-5, depending on dataset size and desired convergence)
        * `per_device_train_batch_size` (e.g., 4-16, depending on memory)
        * `gradient_accumulation_steps` (if needed for larger effective batch size)
        * `learning_rate` (e.g., 2e-4, 5e-5)
        * `logging_steps`, `save_steps`
        * `evaluation_strategy="epoch"` (or `"steps"`)
        * `load_best_model_at_end=True`
    * Justify your choices for key training arguments.

2.  **Create Data Collator:**
    * Use `DataCollatorForLanguageModeling` (with `mlm=False` for causal language modeling).

3.  **Create Trainer and Train:**
    * Instantiate the `Trainer` class with your `peft_model`, `TrainingArguments`, tokenized datasets, and data collator.
    * Start the training process using `trainer.train()`.
    * Show the training loss and evaluation loss (if applicable) during training.

4.  **Qualitative Evaluation:**
    * After training, use the fine-tuned `peft_model` to generate text.
    * Provide at least **3 distinct prompts** related to your dataset's domain.
    * For each prompt, generate text using the `peft_model.generate()` method (remember to set `max_new_tokens`, `do_sample`, `top_k`, `num_beams` if desired).
    * Compare the generated text with what you would expect given your fine-tuning dataset.
    * Discuss the quality of the generated text and how it reflects the fine-tuning. Does it show signs of adapting to your custom data?

5.  **Save and Load LoRA Adapters (Bonus - 5 Marks):**
    * Save only the LoRA adapters using `peft_model.save_pretrained("my_lora_adapters")`.
    * Load the base GPT model again (without LoRA).
    * Load the saved LoRA adapters onto the base model using `PeftModel.from_pretrained(base_model, "my_lora_adapters")`.
    * Demonstrate text generation with the re-loaded LoRA model to confirm it works correctly.
    * Discuss the advantage of saving only the adapters.

In [ ]:
# Your code for defining training arguments, data collator, Trainer, training, and qualitative evaluation.
# Include generated text samples and analysis.
# (For bonus, add code for saving/loading adapters and discussion.)

## Part 4: Reflection and Future Work (Bonus - 5 Marks)

1.  **LoRA Benefits and Limitations:**
    * Summarize the key benefits of using LoRA for fine-tuning LLMs, especially for smaller models or limited resources.
    * What are potential limitations or scenarios where LoRA might not be the optimal choice?

2.  **Hyperparameter Impact:**
    * Discuss how changing `r` (LoRA rank) and `lora_alpha` might affect the model's performance and computational cost.

3.  **Future Enhancements:**
    * Suggest ways to further improve the LoRA fine-tuning process or the quality of generations.
    * Consider combining LoRA with other PEFT methods or advanced training techniques.

## Submission Guidelines

* Submit this Jupyter Notebook (.ipynb file) with all cells executed and outputs visible.
* Ensure your code is well-commented and easy to understand.
* Provide a `requirements.txt` file listing all dependencies.
* Clearly present all requested code, outputs, and discussions.
* Make sure your notebook runs without errors in the specified environment (ideally with GPU for faster training, but CPU should also work for small models/epochs).